In [63]:
# processing chain
# 1. per run. basic hourly stats + key radial profiles
# 2. per run, interpolate and differencing to get get intermediate steps (offset by half hour)
# 3. per ensemble, linearized intermediate steps, each radial coord now has a row, for convenient filtering
# 4. per ensemble, decay stats

In [69]:
import os, glob
import wrftools as wrf
import wrfproc as wp
import pandas as pd
import numpy as np
import importlib
importlib.reload(wrf)
importlib.reload(wp)

dfpathroot='dfs'
runpath='../run/'

In [70]:

runpat='run_land_*rr*'
runsname='run_land_rr'
nmax=12
runlist=[os.path.basename(p) for p in sorted(glob.glob(os.path.join(runpath,runpat)))]
runlist

['run_land_t03_rr',
 'run_land_t04_rr',
 'run_land_t05_rr',
 'run_land_t06_rr',
 'run_land_t07_rr',
 'run_land_t08_rr',
 'run_land_t09_rr',
 'run_land_t10_rr',
 'run_land_t11_rr',
 'run_land_t12_rr',
 'run_land_t13_rr']

In [17]:
# 1.  per run. basic hourly stats + key radial profiles
for run in runlist:
    print(run)
    wolist=wrf.wolist(run)[:12]
    df=wp.get_run_max_df(wolist,run)
    sname=os.path.join(dfpathroot,'df_' + run + '.pkl')
#     print(df)
    df.to_pickle(sname,protocol=4)

In [47]:
# 2. per run, interpolate and differencing to get get intermediate steps (offset by half hour)
for run in runlist:
    dfpath=os.path.join(dfpathroot,'df_'+run+'.pkl')
    spath=os.path.join(dfpathroot,'df_delta_'+run+'.pkl')
    df=pd.read_pickle(dfpath)
    dfd=wp.create_delta_df(df)
    dfd.to_pickle(spath,protocol=4)

In [66]:
# 3. per ensemble, tabulated intermediate steps, each radial coord now has a row, for convenient filtering
# dfpathlist=[os.path.join(dfpathroot,'df_delta_'+run+'.pkl') for run in runlist]
dflist=[]
for run in runlist:
    dfpath=os.path.join(dfpathroot,'df_delta_'+run+'.pkl')
    dflist.append(pd.read_pickle(dfpath))
tlfs=np.arange(2,13) 

In [71]:
dfn=wp.tabulate_df_ens(dflist,tlfs)
sname=os.path.join(dfpathroot,'df_tab_'+runsname+'.pkl')
dfn.to_pickle(sname,protocol=4) 